In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff


In [2]:
train = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, 
                                                  stratify=train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [5]:
xtrain = xtrain[:50000]
ytrain = ytrain[:50000]

In [6]:
token = text.Tokenizer(num_words=None)
max_len = 1500

In [7]:
token.fit_on_texts(list(xtrain) + list(xvalid))

In [ ]:
xtrain_seq = np.array(token.texts_to_sequences(xtrain))
xvalid_seq = np.array(token.texts_to_sequences(xvalid))


In [ ]:
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

In [ ]:
word_index = token.word_index

In [11]:
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Embedding(len(word_index)+1, 300))
model1.add(tf.keras.layers.SimpleRNN(100))
model1.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 300)         51747600  
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               40100     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 51,787,801
Trainable params: 51,787,801
Non-trainable params: 0
_________________________________________________________________


In [13]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
model1.fit(xtrain_pad, ytrain, epochs=15, batch_size=512, 
           validation_data=(xvalid_pad, yvalid))

Epoch 1/15
49/49 [==============================] - 148s 3s/step - loss: 0.3331 - accuracy: 0.8907 - val_loss: 0.2939 - val_accuracy: 0.9043
Epoch 2/15
49/49 [==============================] - 141s 3s/step - loss: 0.2529 - accuracy: 0.9062 - val_loss: 0.2562 - val_accuracy: 0.9138
Epoch 3/15
49/49 [==============================] - 143s 3s/step - loss: 0.2275 - accuracy: 0.9148 - val_loss: 0.2494 - val_accuracy: 0.9217
Epoch 4/15
49/49 [==============================] - 143s 3s/step - loss: 0.1956 - accuracy: 0.9279 - val_loss: 0.2407 - val_accuracy: 0.9251
Epoch 5/15
49/49 [==============================] - 142s 3s/step - loss: 0.1709 - accuracy: 0.9398 - val_loss: 0.2373 - val_accuracy: 0.9268
Epoch 6/15
49/49 [==============================] - 144s 3s/step - loss: 0.1516 - accuracy: 0.9491 - val_loss: 0.2372 - val_accuracy: 0.9267
Epoch 7/15
49/49 [==============================] - 148s 3s/step - loss: 0.1349 - accuracy: 0.9543 - val_loss: 0.2375 - val_accuracy: 0.9274
Epoch 8/15
49

In [ ]:
embeddings_index = {}
f = open('/kaggle/input/glove840b300dtxt/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [31]:
model2 = Sequential()
model2.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))

model2.add(SimpleRNN(100))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])


In [33]:
model2.fit(xtrain_pad, ytrain, epochs=15, batch_size=512, 
           validation_data=(xvalid_pad, yvalid))

Epoch 1/15
98/98 [==============================] - 137s 1s/step - loss: 0.2390 - accuracy: 0.9131 - val_loss: 0.1635 - val_accuracy: 0.9344
Epoch 2/15
98/98 [==============================] - 132s 1s/step - loss: 0.1779 - accuracy: 0.9401 - val_loss: 0.1590 - val_accuracy: 0.9446
Epoch 3/15
98/98 [==============================] - 131s 1s/step - loss: 0.1762 - accuracy: 0.9396 - val_loss: 0.1718 - val_accuracy: 0.9426
Epoch 4/15
98/98 [==============================] - 131s 1s/step - loss: 0.1652 - accuracy: 0.9364 - val_loss: 0.1581 - val_accuracy: 0.9373
Epoch 5/15
98/98 [==============================] - 132s 1s/step - loss: 0.1536 - accuracy: 0.9429 - val_loss: 0.1697 - val_accuracy: 0.9427
Epoch 6/15
98/98 [==============================] - 132s 1s/step - loss: 0.1485 - accuracy: 0.9445 - val_loss: 0.1443 - val_accuracy: 0.9463
Epoch 7/15
98/98 [==============================] - 132s 1s/step - loss: 0.1686 - accuracy: 0.9431 - val_loss: 0.2835 - val_accuracy: 0.9130
Epoch 8/15
98

In [13]:
model3 = Sequential()
model3.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))

model3.add(LSTM(100))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])



In [14]:
model3.fit(xtrain_pad, ytrain, epochs=12, batch_size=512, 
           validation_data=(xvalid_pad, yvalid))

Epoch 1/12
98/98 [==============================] - 64s 651ms/step - loss: 0.2142 - accuracy: 0.9214 - val_loss: 0.1448 - val_accuracy: 0.9444
Epoch 2/12
98/98 [==============================] - 63s 646ms/step - loss: 0.1327 - accuracy: 0.9474 - val_loss: 0.1239 - val_accuracy: 0.9521
Epoch 3/12
98/98 [==============================] - 63s 641ms/step - loss: 0.1210 - accuracy: 0.9513 - val_loss: 0.1203 - val_accuracy: 0.9534
Epoch 4/12
98/98 [==============================] - 63s 645ms/step - loss: 0.1101 - accuracy: 0.9554 - val_loss: 0.1238 - val_accuracy: 0.9528
Epoch 5/12
98/98 [==============================] - 63s 645ms/step - loss: 0.1056 - accuracy: 0.9572 - val_loss: 0.1153 - val_accuracy: 0.9537
Epoch 6/12
98/98 [==============================] - 63s 644ms/step - loss: 0.0994 - accuracy: 0.9598 - val_loss: 0.1157 - val_accuracy: 0.9552
Epoch 7/12
98/98 [==============================] - 63s 646ms/step - loss: 0.0957 - accuracy: 0.9608 - val_loss: 0.1117 - val_accuracy: 0.9548

In [13]:
model4 = Sequential()
model4.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))

model4.add(GRU(100))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])




In [14]:
model4.fit(xtrain_pad, ytrain, epochs=12, batch_size=512, 
           validation_data=(xvalid_pad, yvalid))

Epoch 1/12
98/98 [==============================] - 57s 580ms/step - loss: 0.2147 - accuracy: 0.9218 - val_loss: 0.1307 - val_accuracy: 0.9495
Epoch 2/12
98/98 [==============================] - 57s 577ms/step - loss: 0.1205 - accuracy: 0.9509 - val_loss: 0.1150 - val_accuracy: 0.9552
Epoch 3/12
98/98 [==============================] - 56s 576ms/step - loss: 0.1080 - accuracy: 0.9559 - val_loss: 0.1115 - val_accuracy: 0.9565
Epoch 4/12
98/98 [==============================] - 57s 577ms/step - loss: 0.1034 - accuracy: 0.9573 - val_loss: 0.1089 - val_accuracy: 0.9573
Epoch 5/12
98/98 [==============================] - 57s 578ms/step - loss: 0.0971 - accuracy: 0.9601 - val_loss: 0.1109 - val_accuracy: 0.9564
Epoch 6/12
98/98 [==============================] - 57s 579ms/step - loss: 0.0934 - accuracy: 0.9614 - val_loss: 0.1107 - val_accuracy: 0.9566
Epoch 7/12
98/98 [==============================] - 57s 580ms/step - loss: 0.0895 - accuracy: 0.9633 - val_loss: 0.1161 - val_accuracy: 0.9559

In [ ]:
model5 = Sequential()
model5.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))

model5.add(Bidirectional(LSTM(100)))
model5.add(Dense(1, activation='sigmoid'))
model5.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model5.fit(xtrain_pad, ytrain, epochs=12, batch_size=128, 
           validation_data=(xvalid_pad, yvalid))

Epoch 1/12
98/98 [==============================] - 103s 1s/step - loss: 0.2022 - accuracy: 0.9267 - val_loss: 0.1389 - val_accuracy: 0.9467
Epoch 2/12
98/98 [==============================] - 102s 1s/step - loss: 0.1285 - accuracy: 0.9485 - val_loss: 0.1219 - val_accuracy: 0.9523
Epoch 3/12
98/98 [==============================] - 102s 1s/step - loss: 0.1168 - accuracy: 0.9530 - val_loss: 0.1269 - val_accuracy: 0.9523
Epoch 4/12
98/98 [==============================] - 102s 1s/step - loss: 0.1110 - accuracy: 0.9562 - val_loss: 0.1225 - val_accuracy: 0.9518
Epoch 5/12
98/98 [==============================] - 103s 1s/step - loss: 0.1052 - accuracy: 0.9573 - val_loss: 0.1136 - val_accuracy: 0.9556
Epoch 6/12
98/98 [==============================] - ETA: 0s - loss: 0.1014 - accuracy: 0.9595